In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Load package 
import numpy as np

<h1> Exercise 1: The Simplest Consumption Model (Cake Eating Problem)

Consider the simplest consumption-saving model:

An economic agent chooses how much cake to eat every period, recieving the per-period utility $u_t(c_t)=\sqrt{c_t}$. The agent is born with $W$ cake and can only eat full slices ($1$ is considered "one slice" of cake)

$$\begin{align*}
    V_1(W) &= \max_{c_1,c_2,\dots,c_T} \{ \sqrt{c_1} +\beta \sqrt{c_2}+\beta^2\sqrt{c_3}+\cdots+\beta^T\sqrt{c_T} \} \\
    & \text{s.t.} \\
    W &= c_1 + c_2+\cdots+c_T = \sum_{t=1}^T {c_t} \\
    c_t &\in \mathbb{N}_0 \ \forall t
\end{align*}$$
for $T=3$, $\beta =0.9$ and $W=5$.

We can express this problem with a $\textbf{Bellman equation}$:

The economic agent trades off instantaneous utility with the value associcated with having a larger cake left in the future.

$$\begin{align*}
    V_t(W_t) &= \max_{c_t \in \mathbb{N}_0, 0\leq c_t \leq W_t} \{ \sqrt{c_t} +\beta V_{t+1}(W_{t+1}) \} \\
    & \text{s.t.} \qquad W_{t+1} = W_t -c_t
\end{align*}$$

Note that in the last period $V_T(W_T) = \max_{0\leq c_T \leq W_T}\{ \sqrt{c_T} \}$ because there is no tomorrow, and it is therefore optimal to eat everything, $c_T = W_T$. This is a general feature of finite horizon models, namely that the optimization problem in the last period is static.

### 1. Solve the model using backwards induction (in notebook): 

In [4]:
# 1. setup 
beta = 0.9
W = 5
T = 3

With $W=5, T=3$ and $c_t \in \mathbb{N}_0$, we define value $V_t(\cdot)$ and policy functions $c_t^*(\cdot)$ on the following grids:

$$\begin{bmatrix}
W_1=0, t=1 & W_2=0, t=2 & W_3 = 0, t=3 \\
W_1=1, t=1 & W_2=1, t=2 & W_3 = 1, t=3 \\
\vdots & \vdots & \vdots \\
W_1=5, t=1 & W_2=5, t=2 & W_3 = 5, t=3 \\
\end{bmatrix}$$

Often we will allocate memory by making arrays full of $\texttt{NaN}$ such that we get an error if we make a mistake somewhere.

Note that for $W_t$, the array index and the number of cake slices left, coindices. This is not the case for $t$ as slicing e.g. $\texttt{[:, 1]}$ returns policy or value functions for all $W_t$ but for the period $t=2$. We could instead use $t=0$ for the first period and $T-1$ for the last period if we wanted it to coindice, like for $W_t$ (sometimes we will do this). This is (generally) something to be cautious of when working with discrete states and choices.

In [5]:
# 2. allocate memory

# Allocate memory for value function
V_bi = np.zeros([W+1,T]) + np.nan # make a (W+1)xT matrix of NaNs

# Allocate memory for policy function
Cstar_bi = np.zeros([W+1,T]) + np.nan # make a (W+1)xT matrix of NaNs

In [6]:
# 3. static problem in last period
Cstar_bi[:,T-1] = np.arange(W+1) # always eat all cake left in last period

V_bi[:,T-1] = np.sqrt(Cstar_bi[:,T-1]) # compute associated value of the last period

In [7]:
# 4. solve
# Loop over periods backwards
for t in reversed(range(T-1)):  # t = 1, 0, we already solved for t = 2
    
    # Loop over states
    for w_i in range(W+1):

        # prepare for grid search over all consumption choices
        c = np.arange(w_i+1) # given the agent is in period t and has W_t, the agent can choose to consume c = 0,1,2,...,W_t
        
        # FILL IN. Hint: Find V_next

        ### SOLUTION ###
        w_next = w_i - c
        V_next = V_bi[w_next, t+1]
        ### SOLUTION ###

        V_guess = np.sqrt(c)+beta*V_next #(w+1) vector of possible values next period
        V_bi[w_i, t] = np.amax(V_guess) #Find the maximum value
        Cstar_bi[w_i, t] = np.argmax(V_guess) #Find the corresponding consumption (in this case equal to the index of the maximum value)
        
print(Cstar_bi)

[[0. 0. 0.]
 [1. 1. 1.]
 [1. 1. 2.]
 [1. 2. 3.]
 [2. 2. 4.]
 [2. 3. 5.]]


### 2. Fill in Exercise_1.py with a function with $\beta$, $W$ and $T$ as inputs

### 3. Import the function from $\texttt{Exercise\_1.py}$ and check that it works 

In [8]:
from Exercise_1 import solve_backwards

beta = 0.9
W = 5
T = 3
C,V = solve_backwards(beta=beta, W=W, T=T)

check that value and policy functions in notebook and Exercise_1.py matches

In [9]:
assert (C == Cstar_bi).all() # policy
assert (V == V_bi).all() # value

### 4. Use the policy function to simulate how $W=5$ resources now can be spend optimally over $T=3$ periods.    

In [10]:
# simple simulation (ssim)
C_ssim = np.empty(T) # Make an empty vector of length T to store optimal consumption in
W_now = W # Set the initial cake size to W

for t in range(T): #Loop forwards over periods
    W_now = int(W_now)   # ensure that W_now is an integer (for slicing/indexing)
    # FILL IN. Hint: 1) use policy function C (found above) 2) compute state transition ("new" W_now)

    ### SOLUTION ###
    C_ssim[t] = C[W_now,t]
    W_now = W_now - C_ssim[t]
    ### SOLUTION ###
    
print(C_ssim)

[2. 2. 1.]


### 5. Solve the model for $T = 10$.

In [11]:
# 1. Setup
beta = 0.9
W = 5
T = 10

# 2. Solve 
C10, V10 = solve_backwards(beta,W,T)

Simulate $C_1$, $C_2$,...,$C_{10}$ using the policy function found in 5) and W=5

In [12]:
# Define function to simulate

def simulate(Cstar,T,W):

    # 1. allocate memory
    C_sim = np.empty(T)
    W_now = W

    # 2. simulate "forwards"
    for t in range(T):
        
        W_now = int(W_now)   # ensure that W_now is an integer (for slicing/indexing)

        # FILL IN. Hint: same procedure as in 4)

        ### SOLUTION ###
        C_sim[t] = Cstar[W_now,t]
        W_now = W_now - C_sim[t]
        ### SOLUTION ###
        
    return C_sim

C_sim = simulate(C10,T,W)  # Call function
        
print('Solution for C with T =',T,'and W =', W , ': C =',C_sim)

Solution for C with T = 10 and W = 5 : C = [1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]


Play around with W, $\beta$ and see the results. Are the results inline with your intuition?